In [ ]:
import hashlib
import time
from collections import defaultdict
from tqdm import tqdm

KMER_LENGTH = 5
TABLE_SIZE = 10000003

def hash_function(kmer):
    """Hash function to hash k-mers into table indices."""
    return int(hashlib.md5(kmer.encode()).hexdigest(), 16) % TABLE_SIZE

def read_fasta(file_path):
    """Reads FASTA file and returns a dictionary of sequences."""
    sequences = {}
    with open(file_path, 'r') as file:
        identifier = None
        seq = []
        for line in file:
            line = line.strip()
            if line.startswith(">"):
                if identifier:
                    sequences[identifier] = ''.join(seq)
                identifier = line[1:]
                seq = []
            else:
                seq.append(line)
        if identifier:
            sequences[identifier] = ''.join(seq)
    return sequences

def generate_kmers(seq, k):
    """Generate k-mers from a given sequence."""
    for i in range(len(seq) - k + 1):
        yield seq[i:i+k]

def pseudoalignment(gene_annotations, rna_seqs):
    """Perform pseudoalignment and count equivalence classes."""
    hash_table = defaultdict(list)
    counts = defaultdict(int)
    
    # Indexing gene annotations
    print("Indexing gene annotations...")
    start_time = time.time()
    for gene_id, seq in tqdm(gene_annotations.items(), desc="Indexing", unit="gene"):
        for kmer in generate_kmers(seq, KMER_LENGTH):
            index = hash_function(kmer)
            hash_table[index].append(gene_id)
    print(f"Indexing completed in {time.time() - start_time:.2f} seconds.")
    
    # Pseudoalignment procedure
    print("Pseudoalignment procedure...")
    start_time = time.time()
    total_reads = len(rna_seqs)
    for seq_id, read in tqdm(rna_seqs.items(), desc="Pseudoalignment", unit="read", total=total_reads):
        seen_equivalence_classes = set()
        for kmer in generate_kmers(read, KMER_LENGTH):
            index = hash_function(kmer)
            if index in hash_table and index not in seen_equivalence_classes and hash_table[index]:
                counts[index] += 1
                seen_equivalence_classes.add(index)
    print(f"Pseudoalignment completed in {time.time() - start_time:.2f} seconds.")
    
    # Output equivalence class counts
    equivalence_class_counts = []
    for index, genes in hash_table.items():
        if genes:  # Ensure the index has associated gene IDs
            equivalence_class_counts.append({
                'counts': counts[index],
                'number_of_items_in_equivalence_class': len(genes),
                'isoforms_in_equivalence_class': ','.join(genes)
            })
    
    return equivalence_class_counts

def main():
    gene_annotations_file = 'chr11_transcriptome.fasta'
    rna_seqs_file = 'reads.fasta'
    
    gene_annotations = read_fasta(gene_annotations_file)
    rna_seqs = read_fasta(rna_seqs_file)
    
    equivalence_class_counts = pseudoalignment(gene_annotations, rna_seqs)
    print("Number of equivalence classes:", len(equivalence_class_counts))
    
    for eq_class in equivalence_class_counts:
        print(f"Counts: {eq_class['counts']}, "
              f"Number of items in equivalence class: {eq_class['number_of_items_in_equivalence_class']}, "
              f"Isoforms in equivalence class: {eq_class['isoforms_in_equivalence_class']}")

if __name__ == "__main__":
    main()


In [ ]:
import subprocess
subprocess.run(["wget", "https://raw.githubusercontent.com/biopython/biopython/master/Doc/examples/ls_orchid.fasta"])
subprocess.run(["pip", "install", "biopython", "tqdm"])

from Bio import SeqIO
from tqdm import tqdm
import itertools

def build_index(transcriptome_file, k):
    """ Build an index from the transcriptome file where each k-mer maps to a set of transcript IDs """
    index = {}
    records = list(SeqIO.parse(transcriptome_file, "fasta"))
    for record in tqdm(records, desc="Building index"):
        seq = str(record.seq)
        for i in range(len(seq) - k + 1):
            kmer = seq[i:i+k]
            if kmer not in index:
                index[kmer] = set()
            index[kmer].add(record.id)
    return index

def pseudoalignment(reads_file, index, k):
    """ Map reads to transcripts based on shared k-mers and determine equivalence classes """
    read_to_transcripts = {}
    records = list(SeqIO.parse(reads_file, "fasta"))
    for record in tqdm(records, desc="Processing reads"):
        read_seq = str(record.seq)
        matched_transcripts = []
        for i in range(len(read_seq) - k + 1):
            kmer = read_seq[i:i+k]
            if kmer in index:
                if not matched_transcripts:
                    matched_transcripts = index[kmer]
                else:
                    matched_transcripts = matched_transcripts.intersection(index[kmer])
        # Only consider reads that map to at least one transcript
        if matched_transcripts:
            read_to_transcripts[record.id] = matched_transcripts

    # Determine equivalence classes
    equivalence_classes = {}
    for transcripts in tqdm(read_to_transcripts.values(), desc="Determining equivalence classes"):
        key = frozenset(transcripts)
        if key in equivalence_classes:
            equivalence_classes[key] += 1
        else:
            equivalence_classes[key] = 1

    return equivalence_classes

# Define file paths and parameters
transcriptome_file = 'transcriptomes.fasta'
reads_file = 'reads.fasta'
k = 21  # Define the k-mer length

# Build the index
index = build_index(transcriptome_file, k)

# Perform pseudoalignment
equivalence_classes = pseudoalignment(reads_file, index, k)

# Print equivalence class counts
print(f'Number of equivalence classes: {len(equivalence_classes)}')
for key, count in equivalence_classes.items():
    print(f'Count: {count}, Transcripts: {";".join(key)}')
